In [573]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import metrics
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings("ignore")

In [548]:
data = pd.read_csv('../data/CUAD_data/translated_data.csv', index_col=0)

In [474]:
stop_words = stopwords.words('russian')
stop_words.extend(["это", 'все', 'весь', 'включая', 'отношения', 'цель', "имя","свой", "время", "любой", "соглашение", 
                   "сторона", "настоящий", "право", "компания", "соответствие", "который", "должный", "мочь", "также"])

In [407]:
filtered_sentence = []

for text in data.processed_text_ru.to_numpy():
    sent = []
    for word in text.split(' ')[0:200]:
        if word not in stop_words:
            sent.append(word)
    filtered_sentence.append(' '.join(sent))


In [108]:
vectorizer = TfidfVectorizer(max_features=512)
vectorizer2 = CountVectorizer(max_features=1024)

In [55]:
X = vectorizer.fit_transform(filtered_sentence)
len(vectorizer.get_feature_names_out())

512

In [109]:
X2 = vectorizer2.fit_transform(filtered_sentence)
len(vectorizer2.get_feature_names_out())

1024

In [550]:
codes, uniques = pd.factorize(data.target)
Y = codes

In [551]:
data['codes'] = codes

In [67]:
import xgboost as xgb


In [68]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [69]:
param = {'max_depth': 5, 'eta': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [100]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=150)

In [101]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=150,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [102]:
y_pred = clf.predict(X_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        11
           1       0.00      0.00      0.00         4
           2       0.33      0.75      0.46         4
           3       0.86      0.86      0.86         7
           4       0.67      1.00      0.80         8
           5       1.00      0.75      0.86         4
           6       0.33      1.00      0.50         1
           7       0.33      0.20      0.25         5
           8       1.00      0.88      0.93         8
           9       0.91      1.00      0.95        10
          10       1.00      1.00      1.00         5
          11       0.83      0.83      0.83         6
          12       0.20      0.11      0.14         9
          13       0.38      1.00      0.55         3
          14       0.67      0.89      0.76         9
          15       0.83      0.83      0.83        12
          16       0.90      0.82      0.86        11
          17       0.50    

In [103]:
y_pred = clf.predict(X_train)
print(metrics.classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00         8
           2       1.00      1.00      1.00        14
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        15
           5       1.00      1.00      1.00        20
           6       1.00      1.00      1.00        10
           7       1.00      1.00      1.00        12
           8       1.00      1.00      1.00        24
           9       1.00      1.00      1.00        21
          10       1.00      1.00      1.00        10
          11       1.00      1.00      1.00        11
          12       1.00      1.00      1.00        19
          13       1.00      1.00      1.00        10
          14       1.00      1.00      1.00        25
          15       1.00      1.00      1.00        21
          16       1.00      1.00      1.00        15
          17       1.00    

In [89]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [104]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [105]:
# Create Dictionary
data_words = [i.split() for i in filtered_sentence]
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View


In [106]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=25, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [93]:
lda_model.print_topics()

[(24,
  '0.000*"блокчейн" + 0.000*"вануату" + 0.000*"джек" + 0.000*"биотехнология" + 0.000*"биоамбер" + 0.000*"совместимый" + 0.000*"смарта-контракт" + 0.000*"интервальный" + 0.000*"издержка" + 0.000*"кислота"'),
 (19,
  '0.000*"блокчейн" + 0.000*"вануату" + 0.000*"джек" + 0.000*"биотехнология" + 0.000*"биоамбер" + 0.000*"совместимый" + 0.000*"смарта-контракт" + 0.000*"интервальный" + 0.000*"издержка" + 0.000*"кислота"'),
 (20,
  '0.000*"раздел" + 0.000*"органический" + 0.000*"джек" + 0.000*"биотехнология" + 0.000*"биоамбер" + 0.000*"совместимый" + 0.000*"смарта-контракт" + 0.000*"интервальный" + 0.000*"издержка" + 0.000*"блокчейн"'),
 (11,
  '0.011*"индия" + 0.009*"индийский" + 0.007*"непубличный" + 0.000*"центр" + 0.000*"жидкость" + 0.000*"прокатка" + 0.000*"развлекательный" + 0.000*"реальность" + 0.000*"виртуальный" + 0.000*"геймификация"'),
 (9,
  '0.033*"транзакция" + 0.022*"загрузка" + 0.015*"вместо" + 0.010*"опубликовать" + 0.000*"потребитель" + 0.000*"пин-код" + 0.000*"интеракт

In [94]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -9.688944317069607

Coherence Score:  0.4073895304346442


In [107]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                        x                   y  topics  cluster       Freq
topic                                                                    
16    -0.311937+0.000000j  0.093408+0.000000j       1        1  34.170681
15    -0.259291+0.000000j  0.075203+0.000000j       2        1  16.041469
1     -0.238757+0.000000j -0.043602+0.000000j       3        1   8.391980
12    -0.176260+0.000000j -0.240750+0.000000j       4        1   6.432946
0     -0.273904+0.000000j  0.005786+0.000000j       5        1   6.090315
2     -0.104811+0.000000j -0.166985+0.000000j       6        1   5.782300
4     -0.122852+0.000000j -0.221363+0.000000j       7        1   5.453709
13    -0.051073+0.000000j -0.085216+0.000000j       8        1   3.210755
8     -0.142006+0.000000j  0.173012+0.000000j       9        1   3.142411
7     -0.105764+0.000000j  0.159415+0.000000j      10        1   2.596228
21    -0.165573+0.000000j  0.140219+0.000000j      11        1   2.310084
17    -

In [115]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import StratifiedKFold, GridSearchCV, ParameterGrid
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

import numpy as np

In [116]:
rand_state = 21

In [117]:
pipe = Pipeline(steps=[('vect', CountVectorizer()),
                        ('est', LogisticRegression())])

In [122]:
param_grid = [
  {'vect': [CountVectorizer(), TfidfVectorizer()],
   'vect__min_df': [2, 4, 6],
   'vect__max_df': [0.9, 0.95],
   'vect__ngram_range': [(1, 1), (1, 2)],
   'est': [LogisticRegression()],
   'est__C': np.logspace(-3, 2, 6),
   'est__class_weight': [None, 'balanced'],
   'est__random_state': [rand_state]},
  {'vect': [CountVectorizer(), TfidfVectorizer()],
   'vect__min_df': [2, 4, 6],
   'vect__max_df': [0.9, 0.95],
   'vect__ngram_range': [(1, 1), (1, 2)],
   'est': [RandomForestClassifier(), XGBClassifier()],
   'est__n_estimators': [100, 250, 500],
   'est__class_weight': [None, 'balanced'],
   'est__max_depth': [3, 5, 7]},
  {'vect': [CountVectorizer()],
   'vect__min_df': [2, 4, 6],
   'vect__max_df': [0.9, 0.95],
   'vect__ngram_range': [(1, 1), (1, 2)],
   'est': [GaussianNB()]}
 ]


In [123]:
skf = StratifiedKFold(n_splits=3,
                      shuffle=True,
                      random_state=rand_state)

In [124]:
gs = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring='f1_macro', n_jobs=-1, cv=skf, verbose=1, return_train_score=True)


In [125]:
gs.fit(filtered_sentence, Y)

Fitting 3 folds for each of 1164 candidates, totalling 3492 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=21, shuffle=True),
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('est', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'est': [LogisticRegression()],
                          'est__C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                          'est__class_weight': [None, 'balanced'],
                          'est__random_state': [21],
                          'vect': [CountVecto...
                          'vect': [CountVectorizer(max_df=0.9, min_df=2,
                                                   ngram_range=(1, 2)),
                                   TfidfVectorizer()],
                          'vect__max_df': [0.9, 0.95],
                          'vect__min_df': [2, 4, 6],
                          'vect__ngram_range': [(1, 1), (1, 2)]},
                         {'est': [GaussianNB()], 'vect': [CountVectorizer()],
                          'vect__max_df': [0.9, 0.95],
                          'vect__min_df': [2, 4, 6],
                          'vect__ngram_range': [(1, 1), (1, 2)]}],
             return_train_score=True, scoring='f1_macro', verbose=1)

In [127]:
gs.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2))),
                ('est',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               class_weight=None, colsample_bylevel=1,
                               colsample_bynode=1, colsample_bytree=1,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_threshold=64, max_cat_to_onehot=4,
                               max_delta_step=0, max_depth=7, max_leaves=0,
                               min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=0, num_parallel_tree=1,
                               objective='multi:softprob', ...))])

In [210]:
gs.best_params_

{'est': XGBClassifier(base_score=None, booster=None, callbacks=None, class_weight=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=7, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               predictor=None, ...),
 'est__class_weight': None,
 'est__max_depth': 7,
 'est__n_estimators': 100,
 'vect': CountVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2)),
 'vect__max_df': 0.9,
 'vect__min_df': 2,
 'vect__ngram_range': (1, 2)}

In [129]:
gs.best_score_

0.7779524256522506

In [ ]:
   'est__reg_lambda': [5, 10, 100],
   'est__reg_alpha': [5, 10, 100],
    'est__learning_rate': [0.05, 0.01, 0.1, 0.5]

In [533]:

param_grid1 = [
  {'vect': [CountVectorizer()],
   'vect__min_df': [2],
   'vect__max_df': [0.9],
   'vect__max_features': [1024],
   'vect__ngram_range': [(1, 1)],
   'est': [XGBClassifier()],
   'est__n_estimators': [100],
   'est__class_weight': [None],
   'est__max_depth': [7],
   'est__reg_lambda': [0.01, 0.05, 0.1, 0.5, 0, 1, 3],
   'est__reg_alpha': [0.01, 0.05, 0.1, 0.5, 0, 1, 3],
   'est__learning_rate': [0.01, 0.1, 0.5, 1]
   }
 ]


In [534]:
gs1 = GridSearchCV(estimator=pipe, param_grid=param_grid1, scoring='f1_macro', n_jobs=-1, cv=skf, verbose=2, return_train_score=True)


In [535]:
gs1.fit(filtered_sentence, Y)

Fitting 3 folds for each of 196 candidates, totalling 588 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=21, shuffle=True),
             estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('est', LogisticRegression())]),
             n_jobs=-1,
             param_grid=[{'est': [XGBClassifier(base_score=None, booster=None,
                                                callbacks=None,
                                                class_weight=None,
                                                colsample_bylevel=None,
                                                colsample_bynode=None,
                                                colsample_bytree=None,
                                                early_stopping_round...
                          'est__learning_rate': [0.01, 0.1, 0.5, 1],
                          'est__max_depth': [7], 'est__n_estimators': [100],
                          'est__reg_alpha': [0.01, 0.05, 0.1, 0.5, 0, 1, 3],
                          'est__reg_lambda': [0.01, 0.05, 0.1, 0.5, 0, 1, 3],
                          'vect': [CountVectorizer(max_df=0.9,
                                                   max_features=1024,
                                                   min_df=2)],
                          'vect__max_df': [0.9], 'vect__max_features': [1024],
                          'vect__min_df': [2], 'vect__ngram_range': [(1, 1)]}],
             return_train_score=True, scoring='f1_macro', verbose=2)

In [536]:
gs1.best_score_

0.7774630863470545

In [537]:
gs1.best_params_

{'est': XGBClassifier(base_score=None, booster=None, callbacks=None, class_weight=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=7, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               n_estimators=100, n_jobs=None, num_parallel_tree=None,
               predictor=None, ...),
 'est__class_weight': None,
 'est__learning_rate': 0.1,
 'est__max_depth': 7,
 'est__n_estimators': 100,
 'est__reg_alpha': 0.01,
 'est__reg_lambda': 0.05,
 'vect': CountVectorizer(max_df=0.9, max_features=1024, min_df=2),
 'vect__ma

In [541]:
print(metrics.classification_report(Y, gs1.predict(filtered_sentence)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        32
           1       1.00      1.00      1.00        12
           2       1.00      1.00      1.00        18
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        23
           5       1.00      1.00      1.00        24
           6       1.00      1.00      1.00        11
           7       1.00      1.00      1.00        17
           8       1.00      1.00      1.00        32
           9       1.00      1.00      1.00        31
          10       1.00      1.00      1.00        15
          11       1.00      1.00      1.00        17
          12       1.00      1.00      1.00        28
          13       1.00      1.00      1.00        13
          14       1.00      1.00      1.00        34
          15       1.00      1.00      1.00        33
          16       1.00      1.00      1.00        26
          17       1.00    

In [577]:
vectorizer = CountVectorizer(min_df=2, max_df=0.9, ngram_range=(1,2))
X = vectorizer.fit_transform(filtered_sentence).toarray()

X_train,  X_test, y_train, y_test = train_test_split(X, Y, test_size=1/3, random_state=42)



In [578]:
len(vectorizer.get_feature_names_out())

10065

In [579]:
vectorizer.get_feature_names_out()

array(['ааа', 'ааа иметь', 'абзац', ..., 'японский корпорация', 'ярлык',
       'ящик'], dtype=object)

In [581]:
clf = xgb.XGBClassifier(alpha=1, reg_lambda=0, learning_rate=0.01)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [582]:
print(metrics.classification_report(y_test, y_pred))
print(metrics.classification_report(y_train, clf.predict(X_train)))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       1.00      0.25      0.40         4
           2       0.62      1.00      0.77         5
           3       1.00      0.89      0.94         9
           4       0.67      0.67      0.67         9
           5       1.00      0.67      0.80         6
           6       0.50      1.00      0.67         1
           7       0.33      0.40      0.36         5
           8       1.00      0.89      0.94         9
           9       0.92      1.00      0.96        11
          10       1.00      1.00      1.00         6
          11       1.00      0.67      0.80         6
          12       0.43      0.33      0.38         9
          13       1.00      0.75      0.86         4
          14       0.67      1.00      0.80        12
          15       0.77      0.83      0.80        12
          16       0.77      0.91      0.83        11
          17       0.60    

In [553]:
category_to_id = {}
for idx, i in enumerate(uniques):
    category_to_id[i] = idx

In [556]:
from sklearn.feature_selection import chi2

In [595]:
# Finding the three most correlated terms with each of the product categories
N = 10

for Product, category_id in sorted(category_to_id.items()):
    with open(f'{Product}.txt', 'w') as f:
        features_chi2 = chi2(X, Y == category_id)
        indices = np.argsort(features_chi2[0])
        feature_names = np.array(vectorizer.get_feature_names_out())[indices]
        unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
        bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
        for v in unigrams[-N:]:
            f.write(f'{v}\n')
        for v in bigrams[-N:]:
            f.write(f'{v}\n')
            
        print("\n==> %s:" %(Product))
        print(unigrams[-N:])
print(bigrams[-N:])


==> Affiliate:
['аффилировать', 'нижеподписавшийся', 'офисный', 'элемент', 'сеть', 'сетевой', 'программа', 'слияние', 'партнёр', 'партнёрский']

==> Agency:
['вкладчик', 'правомочный', 'сберегательный', 'федеральный', 'банк', 'агент', 'агентский', 'подписка', 'акция', 'предложение']

==> Co_Branding:
['онлайн', 'сервис', 'центр', 'карьера', 'брендинговый', 'сайт', 'ко', 'кобрендинговый', 'брендинг', 'кобрендинг']

==> Collaboration:
['электроэнергия', 'комитет', 'прийти', 'мвт', 'хайдянь', 'собрание', 'проект', 'платформа', 'совет', 'сотрудничество']

==> Consulting:
['кредиторский', 'эльзас', 'предварительно', 'услуга', 'консультация', 'подрядчик', 'командировочный', 'евро', 'консультационный', 'консультант']

==> Development:
['ферментация', 'биотехнология', 'комбинированный', 'статья', 'тз', 'застройщик', 'антитело', 'коммерциализация', 'разработчик', 'разработка']

==> Distributor:
['принципал', 'скидка', 'напечатать', 'назначать', 'продукция', 'эксклюзивный', 'территория', 'проду

In [586]:
models = [
    RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# 5 Cross-validation
CV = 2
cv_df = pd.DataFrame(index=range(CV * len(models)))

entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, X, Y, scoring='accuracy', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

In [587]:
mean_accuracy = cv_df.groupby('model_name').accuracy.mean()
std_accuracy = cv_df.groupby('model_name').accuracy.std()

acc = pd.concat([mean_accuracy, std_accuracy], axis= 1, 
          ignore_index=True)
acc.columns = ['Mean Accuracy', 'Standard deviation']
acc

,Mean Accuracy,Standard deviation
model_name,,
LinearSVC,0.688235,0.052686
LogisticRegression,0.652941,0.030503
MultinomialNB,0.547059,0.002773
RandomForestClassifier,0.598039,0.002773


In [585]:
# Classification report
print('\t\t\t\tCLASSIFICATIION METRICS\n')
print(metrics.classification_report(y_train, clf.predict(X_train), 
                                    target_names= uniques))

				CLASSIFICATIION METRICS

                    precision    recall  f1-score   support

       Distributor       0.95      0.95      0.95        20
         Promotion       1.00      1.00      1.00         8
            Supply       1.00      1.00      1.00        13
           Hosting       1.00      0.91      0.95        11
     Joint Venture       0.93      1.00      0.97        14
       Endorsement       0.95      1.00      0.97        18
        Consulting       1.00      0.90      0.95        10
         Marketing       0.91      0.83      0.87        12
Strategic Alliance       1.00      1.00      1.00        23
       Sponsorship       0.95      0.95      0.95        20
         Franchise       1.00      1.00      1.00         9
                IP       1.00      1.00      1.00        11
           Service       0.90      0.95      0.92        19
            Agency       1.00      1.00      1.00         9
       Maintenance       0.88      0.95      0.91        22
          